# Flight Agent: Your Travel Planner

Time for an exciting use case. Now is the time to use all the knowledge we have gathered so far to build a complete AI Agent ourselves.

## Plan of Attack:

1. Import libraries
2. Define our tools
3. Define our tools schema
4. Call the OpenAI Responses API
5. Handle tool calls
6. Chat logic
7. Gradio Interface

## Step 1: Import libraries

In [1]:
import os
from tavily import TavilyClient
from dotenv import load_dotenv
import json
from openai import OpenAI
from utils import function_to_tool
from IPython.display import display, Markdown
import gradio as gr

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise ValueError("TAVILY_API_KEY is not set in the environment variables.")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

tavily_client = TavilyClient()
openai_client = OpenAI()

## Step 2: Define our tools

In [ ]:
query = "I want to travel to tokyo and osaka tomorrow what are the best flights available"

# Flight search tool
response = tavily_client.search(
    query=query,
    include_domains = ["skysnanner.com", "expedia.com", "booking.com"]
)

print("Search Results: \n", json.dumps(response, indent=2))

In [ ]:
print(response["results"][0]['content'])

In [2]:
def flight_search(query: str) -> dict:
    """
    Search for flights based on the provided query and optional domains.
    
    Args:
        query (str): The search query for flights.
        include_domains (list, optional): List of domains to include in the search.
        
    Returns:
        dict: Search results containing flight information.
    """
    response = tavily_client.search(
        query=query,
        include_domains=["skyscanner.com", "expedia.com", "booking.com"]
    )
    return response

In [ ]:
query = "What are the best hotels in tokyo and osaka for tomorrow?"

# Hotel search tool
response = tavily_client.search(
    query=query,
    include_domains = ["booking.com", "airbnb.com"]
)

print("Search Results: \n", json.dumps(response, indent=2))

In [3]:
def hotel_search(query: str) -> dict:
    """
    Search for hotels based on the provided query and optional domains.
    
    Args:
        query (str): The search query for hotels.
        include_domains (list, optional): List of domains to include in the search.
        
    Returns:
        dict: Search results containing hotel information.
    """
    response = tavily_client.search(
        query=query,
        include_domains=["booking.com", "airbnb.com"]
    )
    return response

## Step 3: Define our tools schema

In [4]:
flight_tool_schema = function_to_tool(flight_search)
hotel_tool_schema = function_to_tool(hotel_search)

In [ ]:
hotel_tool_schema

## Step 4: Call the OpenAI Responses API

In [5]:
system_message = """

You are a helpful travel planner and your are to assist the user in planning their travel itinerary.

You are to ask the user in a conversational manner about their travel plans and preferences, first start with their
destination, wait for their response and then ask about the dates they plan to travel, and then search for the best flights available,
build an itinerary based on their preferences and give them a summary of their travel plans.


Eg.
AI Assistant: Hello, I am your travel planner. Which country would you like to travel to?
User: I want to travel to Japan
AI Assistant: What is your preferred destination in Japan?
User: Tokyo
AI Assistant: Great! When do you plan to travel?
User: I plan to travel tomorrow.

* use your search tool to search for flight deals *

* You will be provided with flight options *

* use your search tool to search for hotel booking deals *

* You will be provided the results of the hotel search *

summarize results in a neatly formatted itinerary.

provide the itinerary to the user

AI Assistant: Here is your travel itinerary for Japan:

-- Example Itinerary --

Remember to only provide the user with the results once you have information about the flights as well as the hotels.
"""

In [ ]:
input_list = [{"role": "system", "content": system_message}]

def chat(input_list):
    response = openai_client.responses.create(
        model="gpt-4o-mini",
        input=input_list,
        tools=[flight_tool_schema, hotel_tool_schema],
        tool_choice="auto",
        parallel_tool_calls=False
    )

    if response.output[0].type == "message":
        input_list.append({"role": "assistant", "content": response.output_text})
    if response.output[0].type == "function_call":
        input_list += response.output
    return response

response = chat(input_list)
print(response.output)

In [ ]:
input_list

In [ ]:
input_list.append({"role": "user", "content": "TOmorrow"})

response = chat(input_list)

print(response.output)

## Step 5: Handle tools calls

In [6]:
def call_function(name, args):
    if name == "flight_search":
        return flight_search(**args)
    if name == "hotel_search":
        return hotel_search(**args)

In [ ]:
# Save function call outputs for subsequent requests

name = response.output[0].name
args = json.loads(response.output[0].arguments)

result = call_function(name, args)

input_list.append({
    "type": "function_call_output",
    "call_id": response.output[0].call_id,
    "output": str(result)
})


In [ ]:
input_list

In [ ]:
response = chat(input_list)

print(response.output)

In [ ]:
display(Markdown(response.output_text))

## Step 6: Gradio UI

In [ ]:
def chat(message, history):

    clean_history = [{"role": m["role"], "content": m["content"]} for m in history]

    messages = [{"role": "system", "content": system_message}] + clean_history + [{"role": "user", "content": message}]

    response = openai_client.responses.create(
        model="gpt-4o-mini",
        input=messages,
        tools=[flight_tool_schema, hotel_tool_schema],
        tool_choice="auto",
        parallel_tool_calls=False
    )

    if response.output[0].type == "function_call":
        while response.output[0].type == "function_call":
            messages += response.output
            
            name = response.output[0].name
            args = json.loads(response.output[0].arguments)

            result = call_function(name, args)

            messages.append({
                "type": "function_call_output",   # or role="tool" if you want proper chat-style
                "call_id": response.output[0].call_id,
                "output": str(result)
            })

            response = openai_client.responses.create(
                model="gpt-4o-mini",
                input=messages,   # pass full conversation
                tools=[flight_tool_schema, hotel_tool_schema],
                tool_choice="auto",
                parallel_tool_calls=False
            )

    return response.output_text

view = gr.ChatInterface(fn=chat, type="messages").launch()
